In [ ]:
import numpy as np
import math
import cv2
import matplotlib.pyplot as plt
import torch
from torch.utils.data import TensorDataset, DataLoader
from torchvision import datasets, transforms

In [ ]:
epochs = 18
IMG_RESIZE = 500
IMG_CROP = 500
BATCH_SIZE = 16

In [ ]:
import torch.nn as nn
#import torch
class Model(nn.Module):
    def __init__(self, num_classes = 2):
        print("privietsky")
        super(Model, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=10, stride=2, padding=1)
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=3)

        self.conv2 = nn.Conv2d(in_channels=12, out_channels=20, kernel_size=5, stride=2, padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=3)

        self.conv3 = nn.Conv2d(in_channels=20, out_channels=30, kernel_size=3, stride=1, padding=1)
        self.relu3 = nn.ReLU()
        #self.pool3 = nn.MaxPool2d(kernel_size=4)
        
        self.conv4 = nn.Conv2d(in_channels=30, out_channels=30, kernel_size=3, stride=1, padding=1)
        self.relu4 = nn.ReLU()
        
        self.conv5 = nn.Conv2d(in_channels=30, out_channels=30, kernel_size=3, stride=1, padding=1)
        self.relu5 = nn.ReLU()
        self.pool5 = nn.MaxPool2d(kernel_size=3)


        self.fc1 = nn.Linear(in_features=480, out_features=100)
        self.fc_relu = nn.ReLU()
        self.fc2 = nn.Linear(in_features=100, out_features=10)
        #self.fc_relu = nn.ReLU()
        self.fc3 = nn.Linear(in_features=10, out_features=num_classes)

    def forward(self, inp):
        output = self.conv1(inp)
        output = self.relu1(output)
        output = self.pool1(output)

        output = self.conv2(output)
        output = self.relu2(output)
        output = self.pool2(output)

        output = self.conv3(output)
        output = self.relu3(output)
        #output = self.pool3(output)
        
        output = self.conv4(output)
        output = self.relu4(output)
        
        output = self.conv5(output)
        output = self.relu5(output)
        output = self.pool5(output)
        
        output = output.view(len(inp), 480)
        
        output = self.fc1(output)
        output = self.fc_relu(output)
        output = self.fc2(output)
        output = self.fc_relu(output)
        output = self.fc3(output)


        return output

In [ ]:
model = Model(num_classes = 2)
#nn_model.init(num_classes = 2)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr = 0.005)

In [ ]:
loss_data = []
validation_loss = []

#training of network
for epoch in range(epochs):
    print('\rEpoch: [{}/{}]'.format(epoch+1, epochs),end='')
    epoch_loss = 0
    epoch_validation_loss = 0
    for batch_nr, (images, labels) in enumerate(train_loader):
        # clear-the-gradients-of-all-optimized-variables
        optimizer.zero_grad()
        # forward-pass: compute-predicted-outputs-by-passing-inputs-to-the-model
        predict = model.forward(images)
        # calculate-the-batch-loss
        loss = criterion(predict, labels)
        epoch_loss +=loss
        # backward-pass: compute-gradient-of-the-loss-wrt-model-parameters
        loss.backward()
         #perform-a-ingle-optimization-step (parameter-update)
        optimizer.step() 
         #update-training-loss
            
    # Validation
    for batch_nr, (images, labels) in enumerate(val_loader):
        predict = model.forward(images)
        val_loss = criterion(predict, labels)
        epoch_validation_loss += val_loss

    loss_data.append(epoch_loss/len(train_loader))
    validation_loss.append(epoch_validation_loss)

In [ ]:
plt.plot(range(1,epochs+1), loss_data,label='training')
plt.plot(range(1,epochs+1),validation_loss, label='validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
#Testing 
correct = 0
total = 0
confusion = np.array([[0,0],[0,0]])
for batch_nr, (images, labels) in enumerate(test_loader):
    print('\rBatch: [{}/{}]'.format(batch_nr+1, len(test_loader)),end='')
    predict = model.forward(images)
    for i in range(len(images)):

        preede = torch.argmax(predict[i])
        is_correct = preede == labels[i]
        
        if(preede == labels[i]):
            correct +=1
        total += 1
        if(labels[i] == 0):
            if(is_correct):
                confusion[0][0] += 1
            else:
                confusion[1][0] += 1
        if(labels[i] == 1):
            if(is_correct):
                confusion[1][1] += 1
            else:
                confusion[0][1] += 1

print("\nCorrect : ", correct)
print("Total : ", total)
print("Accuracy : " , correct/total*100 , "%")

#plt.imshow(images[0][0])
print(confusion)
